# Detecting Planes from Satellite Images
- Dataset: https://www.kaggle.com/datasets/rhammell/planesnet
- 32,000 Images
- 200x200 Size
- Filename Format: {label} _ {scene id} _ {longitude} _ {latitude}.png
- label: 1 = plane, 0 = no-plane

# Setup

In [97]:
# pip install ultralytics
from ultralytics import YOLO
import pathlib
import tensorflow as tf
from shutil import copyfile
from os import makedirs, listdir, path
import numpy as np

AUTOTUNE = tf.data.AUTOTUNE

In [98]:
ROOT = 'data/plane_sat_cv/'
NO_PLANE = ROOT + 'split_data/no_plane/'
TEST = ROOT + 'yolo/test/'
TRAIN = ROOT + 'yolo/train/'
VALID = ROOT + 'yolo/valid/'
SCENES = ROOT + 'scenes/'
OUT = ROOT + 'yolo/'

IMG_1 = SCENES + 'scene_1.png'
IMG_2 = SCENES + 'scene_2.png'
IMG_3 = SCENES + 'scene_3.png'
IMG_4 = SCENES + 'scene_4.png'

In [3]:
# Create subdirs
def create_dirs(path):
    dirs = ['images/', 'labels/']
    for dir in dirs:
        newdir = path + dir
        makedirs(newdir, exist_ok=True)

In [ ]:
# Create subdirs for train and valid
create_dirs(TRAIN)
create_dirs(VALID)

# Inital Testing
Predicting with pre-trained model. No objects detected in the four scenes.

In [ ]:
# Load a model
base_model = YOLO('yolov8n.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
base_results = base_model([IMG_1, IMG_2, IMG_3, IMG_4])  # return a list of Results objects

# Process results list
for result in base_results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs

    # print(boxes, masks, keypoints, probs, obb)
    
    # display to screen
    # result.show()
    # save to disk
    result.save(filename = OUT + 'result.jpg')

# Attempt to Create Model With Manual Annotations
- Annotated three scenes manually using Roboflow
- Testing on all four

- No Detections

In [ ]:
# pip install roboflow

In [ ]:
from roboflow import Roboflow

API_KEY = 'e8cysaNBvhp4SlHcEbn0'

rf = Roboflow(api_key=API_KEY)
project = rf.workspace("testbench-jd1iq").project("sat_planes")
version = project.version(2)
dataset = version.download(model_format="yolov8", location='/home/bauen/datasets/sat_plane_robo/', overwrite=True)

In [ ]:
# Load Pretrained Model
model = YOLO('yolov8n.pt')

# Train Model
results = model.train(data='/home/bauen/datasets/sat_plane_robo/data.yaml', epochs=20, imgsz=640)

In [ ]:
metrics = model.val()

In [ ]:
# Run batched inference on a list of images
# IMG_1, IMG_2, IMG_3, IMG_4
custom_results = model([IMG_1])  # return a list of Results objects

# Process results list
for result in custom_results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs

    # print(boxes, masks, keypoints, probs, obb)
    
    # display to screen
    # result.show()
    # save to disk
    result.save(filename = OUT + 'custom_result.jpg')

# Attempting to train on small images with slightly smaller bounding boxes
- Detects the entire image as plane

## Split into Training/Validation Sets - Create Labels

In [9]:
# Create Labelled File for Plane Images
# Get the plane images
data_dir = pathlib.Path(TEST).with_suffix('')
# Find image count
image_count = len(list(data_dir.glob('*.png')))
image_count

8000

In [11]:
plane_ds = tf.data.Dataset.list_files(str(data_dir/'*'), shuffle=False)
plane_ds = plane_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [13]:
# Output sample to check if everything is correct
for f in plane_ds.take(5):
  print(f.numpy())

b'data/plane_sat_cv/yolo/test/1__20170618_173641_0c38__-118.40925052785828_33.94085609606337.png'
b'data/plane_sat_cv/yolo/test/1__20151029_161054_0b0a__-118.403770832_33.9411002799.png'
b'data/plane_sat_cv/yolo/test/1__20160714_165520_0c59__-118.407234684_33.9393855686.png'
b'data/plane_sat_cv/yolo/test/1__20161003_213745_1_0c74__-118.430001018_33.9438317458.png'
b'data/plane_sat_cv/yolo/test/1__20170619_180820_0f3f__-122.38370291898272_37.61474007266974.png'


In [48]:
# Split into training/validation sets
val_size = int(image_count * 0.2)
train_ds = plane_ds.skip(val_size)
val_ds = plane_ds.take(val_size)

In [18]:
# Print set lengths
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

6400
1600


In [85]:
def handle_file(file_path):
    # Label format: class_name x y width height
    # Set x/y to center and box to 5 px inside edge of img
    # If your boxes are in pixels, divide x_center and width by image width,
    label = "0 0.5 0.5 0.975 0.975"
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    
    return img, label

In [86]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_pairs = train_ds.map(handle_file, num_parallel_calls=AUTOTUNE)
val_pairs = val_ds.map(handle_file, num_parallel_calls=AUTOTUNE)

In [87]:
for img, label in train_pairs.take(1):
  print("Image Shape: ", img)
  print("Label: ", label.numpy())

Image Shape:  tf.Tensor(b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x14\x00\x00\x00\x14\x08\x02\x00\x00\x00\x02\xeb\x8aZ\x00\x00\x03,IDATx\x9c\x05\xc1\tn\x1c7\x10\x00\xc0\xeef\x933\xb3\x87V+i\xb5R\xe2\xa7\xe6)y\x80?e$\x01\xe2\xc06 \xed1\x07\xc9a\x1f\xa9\xc2\xaf\x7f\xfe\xa1j*\xd2Z\x135\x00\xdfnw\xcf\xc7\x17\x91v\xbd|\xa8\x1a3\x03\x11\x07^[\xcdyq\xd7@\xe4\x0e\x88\xc01\xc4\xc8\xa0\x1c\x98\x83\x99\xb7&\xad\xe94O"\xab\xa8\xaa\xa8\x88\xc4\x98\xba\x98\x08\x10\x11\xdd@\xd5\xdc\xc1\xdd\xd9L\x1d\xc0\xdc\xdd\x9d\x08C w(%\x97ZL\r\xdcMUT\xcdLMD\x14\x10\x08)p\x10QV3\x00\x00\x00DB\xa4\x18\xc9\x1d\xa4\t\xb8\x9b)\x02\x01\x91\xaa\xe4bHh\xee\x89\xe30\x0c1u\xcb\xb2p\xdfujf\x0e\x8cH\x88f\xa6"i\x18\xf6\x9b\xcd\xbc\x8c\xd3\x9cU\xdd\x1c\x00\xccU\xd5LM\x1d\x80\xd6\x96\xcb\xcc\x88D\x04\x08h\xaa\x0e`\xe6\xa2\x9a8\xbc>\xbf\x98=}\xfb\xfb\xdb}\x9a\x01\x03\xa2\x83\x03\x00\xac5\xa3;\x10\xb5VXU\xdd\x01\t\xcc\xd4\x1dZ\x93ZK\xad\xd3\xe3\xe3\xf1|\xfa\xfd\xf36\xde\xa7\xbf\xd6\x96C\xe4\xdd\xfe\xd8\xa5\xde\xad\r\xfd&\

In [77]:
def write_pairs(pairs, base):
    idx = 0
    for img, label in pairs:
        # Create Image Path
        img_path = path.join(base + 'images/', '%s.jpg'%idx)
        # Create Label Path
        label_path = path.join(base + 'labels/', '%s.txt'%idx)
        # Write Image
        tf.io.write_file(img_path, img)
        # Write Label
        tf.io.write_file(label_path, label)

        idx += 1

In [88]:
# Write Training Images/Labels
write_pairs(train_pairs, TRAIN)
write_pairs(val_pairs, VALID)

## Train

In [89]:
# Load Pretrained Model
model = YOLO('yolov8n.pt')

# Train Model
results = model.train(data=OUT + 'data.yaml', epochs=20, imgsz=200)

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data/plane_sat_cv/yolo/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

train: Scanning /home/bauen/data-mining/data/plane_sat_cv/yolo/tr


train: New cache created: /home/bauen/data-mining/data/plane_sat_cv/yolo/train/labels.cache


val: Scanning /home/bauen/data-mining/data/plane_sat_cv/yolo/vali


val: New cache created: /home/bauen/data-mining/data/plane_sat_cv/yolo/valid/labels.cache
Plotting labels to /home/bauen/data-mining/runs/detect/train27/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to /home/bauen/data-mining/runs/detect/train27
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20     0.736G    0.02811    0.03035     0.8892         1
                 Class     Images  Instances      Box(P          

                   all       1600       1600          1          1      0.995      0.995



20 epochs completed in 0.229 hours.
Optimizer stripped from /home/bauen/data-mining/runs/detect/train27/weights/last.pt, 6.2MB
Optimizer stripped from /home/bauen/data-mining/runs/detect/train27/weights/best.pt, 6.2MB

Validating /home/bauen/data-mining/runs/detect/train27/weights/best.pt...
Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


/home/bauen/data-mining/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          


                   all       1600       1600          1          1      0.995      0.995
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/bauen/data-mining/runs/detect/train27


## Validation

In [90]:
metrics = model.val()

Ultralytics YOLOv8.2.6 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/bauen/data-mining/data/plane_sat_cv/yolo/vali
                 Class     Images  Instances      Box(P          


                   all       1600       1600          1          1      0.995      0.995
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/bauen/data-mining/runs/detect/train272


## Testing Inference

In [100]:
# Run batched inference on a list of images
# IMG_1, IMG_2, IMG_3, IMG_4
custom_results = model([IMG_1])  # return a list of Results objects

# Process results list
for result in custom_results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs

    # print(boxes, masks, keypoints, probs, obb)
    
    # display to screen
    # result.show()
    # save to disk
    result.save(filename = OUT + 'custom_result.jpg')


0: 224x224 1 planes, 24.0ms
Speed: 2.1ms preprocess, 24.0ms inference, 9.0ms postprocess per image at shape (1, 3, 224, 224)


# Creating composite images with plane and no-plane images
Not finished. The idea was to create composite images by combining majority of no_plane images with a handful of plane images. The labels could then be easily know based on where in the composite image it was placed.

This may work, but the time limit was reached before this could be achieved.

## Split into Training/Validation Sets - Create Labels

In [93]:
# Create Labelled File for Plane Images
# Get the plane images
data_dir = pathlib.Path(TEST).with_suffix('')
no_plane_dir = pathlib.Path(NO_PLANE).with_suffix('')
# Find image count
image_count = len(list(data_dir.glob('*.png')))
image_count

8000

In [94]:
# Get no_plane list
no_plane_list = list(no_plane_dir.glob('*'))
len(no_plane_list)

24000

In [11]:
plane_ds = tf.data.Dataset.list_files(str(data_dir/'*'), shuffle=False)
plane_ds = plane_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [13]:
# Output sample to check if everything is correct
for f in plane_ds.take(5):
  print(f.numpy())

b'data/plane_sat_cv/yolo/test/1__20170618_173641_0c38__-118.40925052785828_33.94085609606337.png'
b'data/plane_sat_cv/yolo/test/1__20151029_161054_0b0a__-118.403770832_33.9411002799.png'
b'data/plane_sat_cv/yolo/test/1__20160714_165520_0c59__-118.407234684_33.9393855686.png'
b'data/plane_sat_cv/yolo/test/1__20161003_213745_1_0c74__-118.430001018_33.9438317458.png'
b'data/plane_sat_cv/yolo/test/1__20170619_180820_0f3f__-122.38370291898272_37.61474007266974.png'


In [48]:
# Split into training/validation sets
val_size = int(image_count * 0.2)
train_ds = plane_ds.skip(val_size)
val_ds = plane_ds.take(val_size)

In [96]:
# Print set lengths
train_len = tf.data.experimental.cardinality(train_ds).numpy()
val_len = tf.data.experimental.cardinality(val_ds).numpy()
print(train_len, val_len)
# Get num of image to create
train_num = train_len / 10
val_num = val_len / 10
print(train_num, val_num)

6400 1600
640.0 160.0


In [86]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_pairs = train_ds.map(handle_file, num_parallel_calls=AUTOTUNE)
val_pairs = val_ds.map(handle_file, num_parallel_calls=AUTOTUNE)

In [85]:
def handle_file(batch):
    # Create new image with the 10 passed planes and 90 random no_planes
    comb_img = np.zeros((2000, 2000, 3), dtype=np.uint8)
    y = 0
    x = 0
    for file_path in batch:
        img = tf.io.read_file(file_path)
        img = tf.io.decode_image(img)
        img = img.numpy()
        comb_img = np.add(comb_img, img)
        # Create Label: iterate by 200 pixels horizontally and by one row height, then add more rows
        # Label format: class_name x y width height
        label_y = ((y + 200) - 100) / 200
        label_x = ((x + 200) - 100) / 200
        label =  f"0 {label_y} {label_x} 1 1"

    

In [ ]:
# Grab 10 images at a time
for i in range(0, train_len, 10):
    batch = train_ds.skip(i).take(10)
    handle_file(batch)
    write_pairs(train_pairs, TRAIN)